In [ ]:
import pandas as pd
from surprise import Dataset, SVD, Reader, accuracy
from surprise.model_selection import cross_validate
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from surprise.model_selection import train_test_split
import pickle
import joblib


In [ ]:
#movies = pd.read_csv("ml-latest-small/movies.csv")
#ratings = pd.read_csv("ml-latest-small/ratings.csv")
movies = pd.read_csv("ml-32m/movies.csv")
movieData = movies

In [ ]:
#movie = movies[movies['movieID'] == 816692]
for index,row in movies.iterrows():
    if(str(row['movieId']) == '109487'):
        print(row['title'])
        break


In [ ]:
#movieData['genres'] = movies['genres'].apply(lambda x: x.split('|'))

links = pd.read_csv("ml-32m/links.csv")
movieData = pd.merge(movieData, links[['movieId', 'imdbId']], on = 'movieId', how='left')


In [ ]:
print(str(movieData['movieId'].nunique()) + " movies")
print(str(len(ratings)) + " ratings")

In [ ]:
def extract_year(title):
    match = re.search(r'\((\d{4})\)', title)
    if match:
        return int(match.group(1))
    else:
        return None
movieData['year'] = movieData['title'].apply(extract_year)


In [ ]:
movieData.head()

In [ ]:

# Adjusting plot size for readability
plt.figure(figsize=(8, 6))

# Creating a histogram with more bins to space out the data
ratings['rating'].hist(bins=50)  # 'edgecolor' adds space between bars

plt.title('Ratings distribution')
plt.xlabel('Rating')
plt.ylabel('Frequency')

# Display the plot
plt.show()


In [ ]:
movieData = pd.read_csv("movieData_yearextracted.csv")        #OR movieData.csv
ratings = pd.read_csv("ml-32m/ratings.csv")

In [ ]:
ratings.tail()


In [ ]:
reader = Reader(rating_scale = (0.5,5))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)


In [ ]:
trainset, testset = train_test_split(data, test_size=0.05)

In [ ]:
algorithm = SVD()

In [ ]:
algorithm.fit(trainset)

In [ ]:
predictions = algorithm.test(testset)

In [ ]:
accuracy.rmse(predictions)

In [ ]:
with open('32m.pkl', 'rb') as file: #wb or rb
   # pickle.dump(algorithm, file)
    model_svd = pickle.load(file)


In [ ]:
#model = model_svd
model = algorithm

In [ ]:
ratings.tail()

In [ ]:
import pandas as pd

def recommend_movies_random(user_id, model, movies, ratings, n, movie_limit):
    # Find movie IDs that the user has already rated
    user_id = int(user_id)  # Convert to int if 'userId' is int in ratings

    # Find movie IDs that the user has already rated
    rated_movie_ids = set(ratings[ratings['userId'] == user_id]['movieId'])
    
    # Randomly sample movies, then filter out those that the user has already rated
    sampled_movies = movies[~movies['movieId'].isin(rated_movie_ids)].sample(n=movie_limit)
    
    # Create a list to hold predictions
    predictions = []

    # Predict ratings for the sampled movies for the given user
    for index, row in sampled_movies.iterrows():
        movie_id = row['movieId']
        predicted_rating = model.predict(user_id, movie_id)
        
        # Store all relevant information
        predictions.append({
            'movieId': movie_id,
            'title': row['title'],
            'genres': row['genres'],
            'imdbId': row['imdbId'],
            'predicted_rating': predicted_rating.est
        })

    # Sort the predictions by predicted rating in descending order
    predictions.sort(key=lambda x: x['predicted_rating'], reverse=True)

    # Get the top n movie recommendations
    top_recommendations = predictions[:n]

    return top_recommendations

# Example usage
user_id = '300000'  # Replace with an actual user ID from your dataset
recommended_movies = recommend_movies_random(user_id, model_svd, movieData, ratings, n=10, movie_limit=10000)

# Print the recommended movies with titles, genres, and IMDB IDs
print("Top movie recommendations for user", user_id)
for movie in recommended_movies:
    print(f"Predicted Rating: {movie['predicted_rating']:.2f} Movie: {movie['title']}")
    #print(f"Title: {movie['title']}, Genres: {movie['genres']}, IMDB ID: {movie['imdbId']}, Predicted Rating: {movie['predicted_rating']:.2f}")
